# **Импорт библиотек**

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KernelDensity
!pip install umap-learn
import umap
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
!pip install umap-learn[plot]
import umap.plot
colors = px.colors.qualitative.Safe
from scipy import stats
!pip install dash
import dash
from dash import dcc, html

# **EDA исходных данных**

In [65]:
df = pd.read_csv('UnitedData.csv')
df

,String,CMC,pCMC,Temperature,Class
0,CC(CO)O,1.090000,-3.037426,23.0,Nonionic
1,C(CO)CO,1.110000,-3.045323,21.5,Nonionic
2,CCC(CO)O,0.550000,-2.740363,22.0,Nonionic
3,CC(CCO)O,0.530000,-2.724276,21.5,Nonionic
4,C(CCO)CO,0.580000,-2.763428,22.0,Nonionic
...,...,...,...,...,...
1262,CCCCCCCCCCCC(=O)NC(CO)C(=O)O,0.001620,2.790485,25.0,Nonionic
1263,CCCCCCCO[C@@H]1[C@H]([C@H]([C@@H]([C@H](O1)CO)...,0.030600,1.514279,25.0,Nonionic
1264,CCCCCCCCCCCCCC(=O)O[C@@]1([C@@H]([C@H]([C@@H](...,0.000026,4.588380,20.0,Nonionic
1265,CCCCCCCCCCOC(=O)CC(O)C[N+](C)(C)C.[Br-],0.012800,1.892790,25.0,Cationic


Рассмотрим распределение целевой величины:

In [66]:
fig1 = px.histogram(df, x='pCMC', color_discrete_sequence=colors,
                   width=600, height=600, title='pCMC of surfactants')
fig1.show()

In [67]:
fig2 = go.Figure()

fig2.add_trace(go.Histogram(x=df['pCMC'][df['Class']=='Anionic'], name='Anionic', histnorm='probability density'))
fig2.add_trace(go.Histogram(x=df['pCMC'][df['Class']=='Nonionic'], name='Nonionic', histnorm='probability density'))
fig2.add_trace(go.Histogram(x=df['pCMC'][df['Class']=='Cationic'], name='Cationic', histnorm='probability density'))
fig2.add_trace(go.Histogram(x=df['pCMC'][df['Class']=='Zwitterionic'], name='Zwitterionic', histnorm='probability density'))

kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(df['pCMC'].values.reshape(-1, 1))
x_d = np.linspace(df['pCMC'].min(), df['pCMC'].max(), 1267)
log_dens = kde.score_samples(x_d.reshape(-1, 1))
density = np.exp(log_dens)

density_normalized = density / np.max(density)

fig2.add_trace(go.Scatter(x=x_d, y=density_normalized, mode='lines', name='KDE', line=dict(color='purple')))

fig2.update_layout(barmode='stack', title_text='Class distribution of pCMC with density estimation', xaxis_title_text='pCMC', height=600, width=700, colorway=colors)
fig2.update_traces(opacity=0.9)
fig2.show()

Видно, что распределение не является нормальным, но проведем статистический тест для проверки:

In [68]:
statistic, p_value = stats.normaltest(df['pCMC'])
print(f"Статистика теста: {statistic}")
print(f"P-значение: {p_value}")

alpha = 0.05
if p_value > alpha:
    print("Данные похожи на нормальное распределение (гипотеза о нормальности не отвергается).")
else:
    print("Данные не похожи на нормальное распределение (гипотеза о нормальности отвергается).")

Статистика теста: 46.18238731112325
P-значение: 9.367463613770246e-11
Данные не похожи на нормальное распределение (гипотеза о нормальности отвергается).


In [69]:
fig3 = px.violin(df, y='pCMC', color='Class', color_discrete_sequence=colors,
                   width=800, height=500, title='pCMC of classes of surfactants', box=True)
fig3.show()

Проведем статистический тест Крускала-Уоллиса, для сравнения разных классов ПАВ. Установим нулевую гипотезу: класс ПАВ не влияет на показатель ККМ.



In [70]:
statistic, p_value = stats.kruskal(df[df['Class'] == 'Anionic']['pCMC'],
                                      df[df['Class'] == 'Cationic']['pCMC'],
                                      df[df['Class'] == 'Zwitterionic']['pCMC'],
                                      df[df['Class'] == 'Nonionic']['pCMC'])

print("Значение статистики:", statistic)
print("Значение p-значения:", p_value)
if p_value > 0.01:
    print("Нет статистически значимой разницы между выборками, верна нулевая гипотеза")
else:
    print("Существует статистически значимая разница между выборками")

Значение статистики: 9.21191305279345
Значение p-значения: 0.026602118759561165
Нет статистически значимой разницы между выборками, верна нулевая гипотеза


In [71]:
fig4 = px.pie(df, values='pCMC', names='Class', color_discrete_sequence=colors,
                   width=600, height=600, title='Class distribution of surfactants')
fig4.show()

In [101]:
fig5 = px.scatter(df, x='pCMC', y='Temperature', color='Class', color_discrete_sequence=colors, size=[10]*len(df), size_max=14)
fig5.show()

Можно заметить, что изменение температуры проведения эксперимента, не оказывает сущесвтенного влияния на показатель концентрации мицеллообразования. Проверим это с помощью корреляционного теста Спирмана:

In [73]:
correlation, p_value = stats.spearmanr(df['pCMC'], df['Temperature'])

print(f"Коэффициент корреляции Спирмена: {correlation}")
print(f"P-значение: {p_value}")

# Оцениваем статистическую значимость корреляции
if p_value > 0.05:  # Уровень значимости 0.05
    print("Нет статистически значимой корреляции")
else:
    print("Корреляция статистически значима")

Коэффициент корреляции Спирмена: 0.1868017969736609
P-значение: 2.0599364697795273e-11
Корреляция статистически значима


Тест показал, что некоторая корреляция есть, но сам коэффициент не высокого значения, и взгляд на график говорит об этом.

Для более качественного построения модели, необходимо не только большое число опытов и данных, но и наличие большого числа рассмотренных уникальных молекул. Рассчитаем, сколько молекул из набора данных уникально:

In [74]:
m1 = df['String'].nunique()
m1

1111

Для проекта разработки ПАВ необходимо, чтобы у молекулы ККМ была как можно меньше, для наших целей подойдет значение CMC=0,01 моль/л и меньше. Рассчитаем сколько данных имеется с молекулами с данным требованием:

In [75]:
m2 = len(df['CMC'][df['CMC'] >= 0.01])/len(df['CMC'])
m2

0.2517758484609313

# **Визуализация пространства молекул с дескрипторами RDKit**

In [102]:
df_rd = pd.read_csv('Data_rdkit.csv')
df_rd

,String,CMC,pCMC,Temperature,Class,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CC(CO)O,1.090000,-3.037426,23.0,Nonionic,8.111111,8.111111,0.138889,-0.560185,0.435835,...,0,0,0,0,0,0,0,0,0,0
1,C(CO)CO,1.110000,-3.045323,21.5,Nonionic,7.906250,7.906250,0.093750,0.093750,0.459049,...,0,0,0,0,0,0,0,0,0,0
2,CCC(CO)O,0.550000,-2.740363,22.0,Nonionic,8.416667,8.416667,0.114583,-0.509259,0.490543,...,0,0,0,0,0,0,0,0,0,0
3,CC(CCO)O,0.530000,-2.724276,21.5,Nonionic,8.392361,8.392361,0.081019,-0.351852,0.490543,...,0,0,0,0,0,0,0,0,0,0
4,C(CCO)CO,0.580000,-2.763428,22.0,Nonionic,8.086250,8.086250,0.195000,0.195000,0.469063,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,CCCCCCCCCCCC(=O)NC(CO)C(=O)O,0.001620,2.790485,25.0,Nonionic,11.438346,11.438346,0.296856,-1.200991,0.454130,...,0,0,0,0,0,0,0,0,8,0
1263,CCCCCCCO[C@@H]1[C@H]([C@H]([C@@H]([C@H](O1)CO)...,0.030600,1.514279,25.0,Nonionic,9.720300,9.720300,0.423021,-1.364823,0.457892,...,0,0,0,0,0,0,0,0,4,0
1264,CCCCCCCCCCCCCC(=O)O[C@@]1([C@@H]([C@H]([C@@H](...,0.000026,4.588380,20.0,Nonionic,12.909030,12.909030,0.124133,-2.826419,0.082810,...,0,0,0,0,0,0,0,0,10,0
1265,CCCCCCCCCCOC(=O)CC(O)C[N+](C)(C)C.[Br-],0.012800,1.892790,25.0,Cationic,11.557246,11.557246,0.000000,-0.614754,0.279950,...,0,0,0,0,0,0,0,0,7,0


Оставим только уникальные молекулы для данной задачи:

In [103]:
df_rd = df_rd.drop_duplicates(subset=['String'])
df_rd.shape

(1111, 207)

In [104]:
df_pca = df_rd.iloc[:,5:]
y = df_rd['pCMC']

scaling=StandardScaler()
scaling.fit(df_pca)
Scaled_data=scaling.transform(df_pca)

# Число компонент ставим равное 2, для визуализации
principal=PCA(n_components=2)
principal.fit(Scaled_data)
x=principal.transform(Scaled_data)

print(x.shape)

(1111, 2)


In [105]:
fig6 = px.scatter(x, x=x[:,0], y=x[:,1],
                color=y,width=600, height=600, title='PCA for RDKit descriptors',
                labels={'x':'PC1','y':'PC2','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Viridis)
fig6.show()

In [106]:
x = df_rd.iloc[:,5:]
pipe = make_pipeline(SimpleImputer(strategy="mean"))
X = pipe.fit_transform(x.copy())

In [107]:
manifold = umap.UMAP().fit(X, y)
X_reduced = manifold.transform(X)

In [108]:
fig7 = px.scatter(X_reduced, x=X_reduced[:, 0], y=X_reduced[:,1],
                color=y,width=600, height=600, title='UMAP for RDKit descriptors',
                labels={'x':'','y':'','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Plotly3)
fig7.show()

In [109]:
pipe = make_pipeline(SimpleImputer(strategy="mean"), QuantileTransformer())
X = pipe.fit_transform(x.copy())

manifold = umap.UMAP().fit(X, y)
X_reduced_2 = manifold.transform(X)

In [110]:
fig8 = px.scatter(X_reduced_2, x=X_reduced_2[:, 0], y=X_reduced_2[:,1],
                color=y,width=600, height=600, title='UMAP Quantile transform for RDKit descriptors',
                labels={'x':'','y':'','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Plotly3)
fig8.show()

In [111]:
pipe = make_pipeline(PowerTransformer())
X = pipe.fit_transform(x.copy())

y_encoded = pd.factorize(y)[0]

manifold = umap.UMAP().fit(X, y_encoded)
X_reduced_3 = manifold.transform(X)

In [112]:
fig9 = px.scatter(X_reduced_3, x=X_reduced_3[:, 0], y=X_reduced_3[:,1],
                color=y,width=600, height=600, title='UMAP Power transform for RDKit descriptors',
                labels={'x':'','y':'','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Viridis)
fig9.show()

Метод снижения размерности t-SNE для визуализации пространства молекул:

In [113]:
features = df_rd.iloc[:,5:]

tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(features)

fig10 = px.scatter(projections, x=0, y=1,
                color=y,width=600, height=600, title='t-SNE for RDKit descriptors',
                labels={'0':'','1':'','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Hot)
fig10.show()

# **Визуализация пространства молекул с отпечатками пальцев RDKit**

In [114]:
df_fp = pd.read_csv('Data_fps.csv')
df_fp

,String,CMC,pCMC,Temperature,Class,Col_0,Col_1,Col_2,Col_3,Col_4,...,Col_2038,Col_2039,Col_2040,Col_2041,Col_2042,Col_2043,Col_2044,Col_2045,Col_2046,Col_2047
0,CC(CO)O,1.090000,-3.037426,23.0,Nonionic,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,C(CO)CO,1.110000,-3.045323,21.5,Nonionic,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CCC(CO)O,0.550000,-2.740363,22.0,Nonionic,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,CC(CCO)O,0.530000,-2.724276,21.5,Nonionic,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,C(CCO)CO,0.580000,-2.763428,22.0,Nonionic,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,CCCCCCCCCCCC(=O)NC(CO)C(=O)O,0.001620,2.790485,25.0,Nonionic,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1263,CCCCCCCO[C@@H]1[C@H]([C@H]([C@@H]([C@H](O1)CO)...,0.030600,1.514279,25.0,Nonionic,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1264,CCCCCCCCCCCCCC(=O)O[C@@]1([C@@H]([C@H]([C@@H](...,0.000026,4.588380,20.0,Nonionic,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1265,CCCCCCCCCCOC(=O)CC(O)C[N+](C)(C)C.[Br-],0.012800,1.892790,25.0,Cationic,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


Оставим только уникальные молекулы для данной задачи:

In [115]:
df_fp = df_fp.drop_duplicates(subset=['String'])
df_fp.shape

(1111, 2053)

In [116]:
df_pca = df_fp.iloc[:,5:]
y = df_fp['pCMC']

scaling=StandardScaler()
scaling.fit(df_pca)
Scaled_data=scaling.transform(df_pca)

# Число компонент ставим равное 2, для визуализации
principal=PCA(n_components=2)
principal.fit(Scaled_data)
x=principal.transform(Scaled_data)

print(x.shape)

(1111, 2)


In [117]:
fig11 = px.scatter(x, x=x[:,0], y=x[:,1],
                color=y,width=600, height=600, title='PCA for fingerprints',
                labels={'x':'PC1','y':'PC2','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Viridis)
fig11.show()

In [118]:
x = df_fp.iloc[:,5:]
pipe = make_pipeline(SimpleImputer(strategy="mean"))
X = pipe.fit_transform(x.copy())

In [119]:
manifold = umap.UMAP().fit(X, y)
X_reduced = manifold.transform(X)

In [120]:
fig12 = px.scatter(X_reduced, x=X_reduced[:, 0], y=X_reduced[:,1],
                color=y,width=600, height=600, title='UMAP for fingerprints',
                labels={'x':'','y':'','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Plotly3)
fig12.show()

In [121]:
pipe = make_pipeline(SimpleImputer(strategy="mean"), QuantileTransformer())
X = pipe.fit_transform(x.copy())

# Fit UMAP to processed data
manifold = umap.UMAP().fit(X, y)
X_reduced_2 = manifold.transform(X)

In [122]:
fig13 = px.scatter(X_reduced_2, x=X_reduced_2[:, 0], y=X_reduced_2[:,1],
                color=y,width=600, height=600, title='UMAP Quantile transform for fingerprints',
                labels={'x':'','y':'','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Plotly3)
fig13.show()

In [123]:
pipe = make_pipeline(PowerTransformer())
X = pipe.fit_transform(x.copy())

y_encoded = pd.factorize(y)[0]

manifold = umap.UMAP().fit(X, y_encoded)
X_reduced_3 = manifold.transform(X)

In [124]:
fig14 = px.scatter(X_reduced_3, x=X_reduced_3[:, 0], y=X_reduced_3[:,1],
                color=y,width=600, height=600, title='UMAP Power transform for fingerprints',
                labels={'x':'','y':'','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Inferno)
fig14.show()

Метод снижения размерности t-SNE для 'отпечатков пальцев' молекул:

In [125]:
features = df_fp.iloc[:,5:]

tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(features)

fig15 = px.scatter(projections, x=0, y=1,
                color=y,width=600, height=600, title='t-SNE for fingerprints',
                labels={'0':'','1':'','color':'pCMC'}, color_continuous_scale=px.colors.sequential.Plasma)
fig15.show()

# **Создание Dashbord-а**

In [126]:
app = dash.Dash(__name__)

# Определение макета приложения
metrici = html.Div([
    html.H1("Метрики молекул"),
    html.H2(f"Число уникальных молекул: {m1}"),
    html.H2(f"Процент молекул с ККМ менее 0.01 моль/л: {m2:.2%}")],
    style={'textAlign': 'left', 'color': 'black', 'fontSize': 10, 'fontFamily': 'Arial, sans-serif'})
first_div = html.Div(style={"display": "flex", 'height': '50%', 'width': '50%'},
                     children=[dcc.Graph(figure=fig2), dcc.Graph(figure=fig4)])
second_div = html.Div(style={"display": "flex", 'height': '50%', 'width': '70%'},
                     children=[dcc.Graph(figure=fig3), dcc.Graph(figure=fig5)])
zag_space = html.Div([html.H1("Визуализация пространства молекул в датасете")], style={'textAlign': 'center', 'color': 'black', 'fontSize': 10, 'fontFamily': 'Arial, sans-serif'})
third_div = html.Div(style={"display": "flex", 'height': '50%', 'width': '50%'},
                     children=[dcc.Graph(figure=fig6), dcc.Graph(figure=fig10), dcc.Graph(figure=fig15)])
app.layout = html.Div([html.H1("Анализ качества данных для предсказания концентрации мицеллообразования"), metrici,
                       first_div, second_div, zag_space , third_div], style={'textAlign': 'center', 'color': 'black', 'fontSize': 15, 'fontFamily': 'Arial, sans-serif'})
# Запуск приложения
if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>